Zaimportowanie bibliotek:

In [17]:
import skimage.io as sk_io
import skimage.feature as sk_ft
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
from os import mkdir, listdir
import shutil
from os.path import exists, join
from numpy import shape, pi, array, concatenate, zeros
from pandas import DataFrame as df
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC as svc
from sklearn.neighbors import NearestNeighbors as nn
from sklearn.metrics import accuracy_score, confusion_matrix

Funkcja wczytująca obrazy tekstur z folderów, wycinająca próbki i zapisująca je do osobnych folderów:

In [3]:
def get_samples(sample_size):
    for i in range(1,5):
        folder_path = (R'./lab3_tekstury/'+str(i)+'/samples')
        if exists(folder_path):
            shutil.rmtree(folder_path)
            mkdir(folder_path)
        else:
            mkdir(folder_path)
        for j in range(1,5):
            photo = sk_io.imread(R'./lab3_tekstury/'+str(i)+'/'+str(j)+'.jpg')
            for height in range(shape(photo)[0]//sample_size):
                for width in range(shape(photo)[1]//sample_size):
                    sample = photo[height*sample_size:(height+1)*sample_size,width*sample_size:(width+1)*sample_size]
                    sk_io.imsave(folder_path+'/sample_'+str(j)+str(height)+str(width)+'.jpg', sample)

get_samples(128)

d:\Programy\Aplikacje\Python_3.11\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ./lab3_tekstury/1/samples/sample_1030.jpg is a low contrast image
  return func(*args, **kwargs)
d:\Programy\Aplikacje\Python_3.11\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ./lab3_tekstury/1/samples/sample_1130.jpg is a low contrast image
  return func(*args, **kwargs)
d:\Programy\Aplikacje\Python_3.11\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ./lab3_tekstury/1/samples/sample_1230.jpg is a low contrast image
  return func(*args, **kwargs)
d:\Programy\Aplikacje\Python_3.11\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ./lab3_tekstury/1/samples/sample_1330.jpg is a low contrast image
  return func(*args, **kwargs)
d:\Programy\Aplikacje\Python_3.11\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: ./lab3_tekstury/1/samples/sample_1430.jpg is a low contrast image
  return func(*args, **kwargs)
d:\Programy\Aplikacje\Python_3.11\Lib\si

Tworzenie zbioru danych. Pętla kolejno wczytuje próbkę tekstury, konwertuje z RGB do skali szarości, zmienia poziom jasności na 6-bitowy, wyznacza macierz zdarzeń i na jej podstawie wylicza odpowiednie parametry. Parametry występują w postaci macierzy 3x4, więc są konwertowane na wektor 72 cech, do których na koniec przypisana zostaje kategoria tekstury. Następnie wektory wszystkich próbek są łączone w jedną zmienną.

In [45]:
features = zeros([1,7])
for i in range(1,5):
    folder_path = (R'./lab3_tekstury/'+str(i)+'/samples')
    print(i)
    for j in range(len(listdir(folder_path))):
        #Wgranie próbki,  zmiana na 6 bitową skalę szarości
        element_path = join(folder_path, listdir(folder_path)[j])
        sample = (rgb2gray(sk_io.imread(element_path)))
        sample_6bit = img_as_ubyte((sample*63)/255)
        #Wyznaczenie macierzy zdarzeń oraz na jej podstawie wybranych cech
        GLCM = sk_ft.graycomatrix(sample_6bit,distances=[1,3,5],angles=[0, pi/8, pi/4, 3*pi/8])
        dissimilarity = sk_ft.graycoprops(GLCM,'dissimilarity')
        correlation = sk_ft.graycoprops(GLCM,'correlation')
        contrast = sk_ft.graycoprops(GLCM,'contrast')
        energy = sk_ft.graycoprops(GLCM,'energy')
        homogeneity = sk_ft.graycoprops(GLCM,'homogeneity')
        ASM = sk_ft.graycoprops(GLCM,'ASM')
        #Zapis próbki z wyznaczonymi cechami listy zbiorczej
        list_features = array([dissimilarity, correlation, contrast, energy, homogeneity, ASM])
        list_features = array(list_features.tolist()).reshape(1, 72)
        sample_features = concatenate([list_features, [[i]]], axis=1)
        if j == 0 and i ==1:
            features = sample_features
        else:
            features = concatenate([features, sample_features], axis=0)

1
2
3
4


Konwersja zbioru danych do DataFrame, a następnie zapis do pliku .csv:

In [47]:
plik = df(features)
plik.to_csv("texture_samples.csv")

Wydzielenie ze zbioru danych cech oraz etykiet, a następnie podzielenie zbioru na treningowy i testowy:

In [55]:
Y = features[:,-1]
X = features[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

Wykonanie klasyfikacji tekstur za pomocą algorytmu wektrów wspierających. Wyznaczenie dokładności klasyfikatora oraz macierzy pomyłek.

In [56]:
classifier = svc()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
conf_matrix = confusion_matrix(y_test,y_pred,normalize="true")
print("model accuracy= ",accuracy)
print("confusion matrix:\n",conf_matrix)

model accuracy=  0.9004959702417855
confusion matrix:
 [[0.7097561  0.00365854 0.28658537 0.        ]
 [0.00493218 0.99506782 0.         0.        ]
 [0.0308642  0.         0.9691358  0.        ]
 [0.         0.06878981 0.         0.93121019]]
